## Importing Packages 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from googleapiclient.discovery import build
import api_config

## Scraping data from Youtube with Api 

In [11]:
channel_id = 'UC5_Q895CBrZLP7hm83uexjg'
youtube = build('youtube', 'v3', developerKey=api_config.yt_Api_key)
youtube_2 = build('youtube', 'v3', developerKey=api_config.yt_Api_key2)

In [1]:
def search_ytchannel_with_keyword(youtube, query='data science | data analyst | data scientist | machine learning'):
    """ This function search the youtube channel base on the keyword entered as the **kwargs

    Args:
        youtube (Youtube Api build): _description_
        query (str, optional): Take in string search keywords. Defaults to 'data science | data analyst | data scientist | machine learning'.

    Returns:
        Json data: list of dicts to parse into dataframe
    """    
    request = youtube.search().list(
        part="snippet",
        maxResults=100,
        q=query,
        type='channel' # requesting for only channels
    )
    response = request.execute()
    channels = [] # empty lsit to stored the iterations
    for i in range(len(response['items'])): # iterating over the first 100 response 
        data = dict(channel_title = response['items'][i]['snippet']['channelTitle'],
                    channel_id = response['items'][i]['snippet']['channelId'])
        channels.append(data)
    next_page_token = response.get('nextPageToken')
    more_pages = True # Condition to end the while loop

    while more_pages:
        if next_page_token is None:
            more_pages = False
        elif len(response['items']) == 1000: # second condition to break the while loop
            break
        else:
            request = youtube.search().list(
            part="snippet",
            maxResults=100,
        q=query,
        type='channel,playlist,videos',
        pageToken = next_page_token
    )
        response = request.execute()

        for i in range(len(response['items'])):
            data = dict(channel_title = response['items'][i]['snippet']['channelTitle'],
                        channel_id = response['items'][i]['snippet']['channelId'])
        
            channels.append(data)
        next_page_token = response.get('nextPageToken')
    
    return channels

In [ ]:
fresh_json_data = search_ytchannel_with_keyword(youtube_2, query='data science | data analyst') # sending the Api request

In [ ]:
# Saving the json file response
file_name = 'channels_2_data.json'
with open(file_name, 'w') as file:
    json.dump(fresh_json_data, file)

In [ ]:
# Another request to to get more data
request = youtube.search().list(
        part="snippet",
        maxResults=100,
        q="data science" or "data analysis" or "SQL" or "machine learning" or "data scientist",
        type='channel,playlist,videos' # requesting for videos, channels and playlist with the keyword
    )
response = request.execute()
channels = []
for i in range(len(response['items'])):
    data = dict(channel_title = response['items'][i]['snippet']['channelTitle'],
                channel_id = response['items'][i]['snippet']['channelId'])
#                 subscribers = response['items'][i]['statistics']['subscriberCount'])

    channels.append(data)
next_page_token = response.get('nextPageToken')
more_pages = True

while more_pages:
    if next_page_token is None:
        more_pages = False
    else:
        request = youtube.search().list(
        part="snippet",
        maxResults=100,
        q="data science" or "data analysis" or "SQL" or "machine learning" or "data scientist",
        type='channel,playlist,videos',
        pageToken = next_page_token
    )
        response = request.execute()

    for i in range(len(response['items'])):
        data = dict(channel_title = response['items'][i]['snippet']['channelTitle'],
                    channel_id = response['items'][i]['snippet']['channelId'])
        
        channels.append(data)
    next_page_token = response.get('nextPageToken')

print(len(channels))

In [ ]:
# Saing the second json file
file_name = 'Yt_data.json'
with open(file_name, 'w') as file:
    json.dump(channels, file)

Reason i request for more data using more keywords is because i search through the json and couldn't find some common data science youtube channels

In [12]:
# Opening the Json
with open('Yt_data.json', 'r') as file:
    youtube = json.load(file)


In [13]:
with open('channels_2_data.json', 'r') as file:
    youtube_1 = json.load(file)

In [14]:
print(len(youtube))
print(len(youtube_1))

602
555


In [15]:
df_1 = pd.DataFrame(youtube)
df_2 = pd.DataFrame(youtube_1)

In [8]:
df_1.head()

,channel_title,channel_id
0,DSN- Data Scientists Network(Data Science Nige...,UCTi2xuVi9CLx9QryIJYLbaQ
1,freeCodeCamp.org,UC8butISFwT-Wl7EV0hUK0BQ
2,codebasics,UCh9nVJoWXmFb7sLApWGcLPQ
3,Steve Brunton,UCm5mt-A4w61lknZ9lCsZtBw
4,Data Professor,UCV8e2g4IWQqK71bbzGDEI4Q


In [16]:
df_2 = df_2.drop_duplicates()

In [17]:
df_1 = df_1.drop_duplicates()

In [18]:
new_df = pd.concat([df_1, df_2])
new_df

,channel_title,channel_id
0,DSN- Data Scientists Network(Data Science Nige...,UCTi2xuVi9CLx9QryIJYLbaQ
1,freeCodeCamp.org,UC8butISFwT-Wl7EV0hUK0BQ
2,codebasics,UCh9nVJoWXmFb7sLApWGcLPQ
3,Steve Brunton,UCm5mt-A4w61lknZ9lCsZtBw
4,Data Professor,UCV8e2g4IWQqK71bbzGDEI4Q
...,...,...
548,Tony Games,UC9SDde41KHNC-VGw0qJ57mQ
549,Sheldon,UCtE0e8G-9zjwrQrug3Mra_A
550,DɾJØҽ༒Pɨυֆ༒Mᗩɳᗩƙɨʅ,UCsPQUCWRjWWGFvdyhjNLJ2A
551,Joshua Pavelko,UCEcc1trh3yFCcXHA1YQOezw


In [13]:
duplicate = new_df[new_df.duplicated()]
duplicate

,channel_title,channel_id
16,365 Data Science,UCEBpSZhI1X8WaP-kY_2LLcg
20,MarinStatsLectures-R Programming & Statistics,UCaNIxVagLhqupvUiDK01Mgg
21,365 Data Science,UCEBpSZhI1X8WaP-kY_2LLcg
24,Data Professor,UCV8e2g4IWQqK71bbzGDEI4Q
41,ritvikmath,UCUcpVoi5KkJmnE3bvEhHR0Q
...,...,...
478,Pacmann AI,UC5oiB8wDG17W8XALdLjlFEw
502,Simplilearn,UCsvqVGtbbyHaMoevxPAq9Fg
514,Techcanvass,UCDHsj-2kb_bGk_Dr3ktSSbg
553,DɾJØҽ༒Pɨυֆ༒Mᗩɳᗩƙɨʅ,UCsPQUCWRjWWGFvdyhjNLJ2A


In [19]:
new_df = new_df.drop_duplicates()
new_df

,channel_title,channel_id
0,DSN- Data Scientists Network(Data Science Nige...,UCTi2xuVi9CLx9QryIJYLbaQ
1,freeCodeCamp.org,UC8butISFwT-Wl7EV0hUK0BQ
2,codebasics,UCh9nVJoWXmFb7sLApWGcLPQ
3,Steve Brunton,UCm5mt-A4w61lknZ9lCsZtBw
4,Data Professor,UCV8e2g4IWQqK71bbzGDEI4Q
...,...,...
548,Tony Games,UC9SDde41KHNC-VGw0qJ57mQ
549,Sheldon,UCtE0e8G-9zjwrQrug3Mra_A
550,DɾJØҽ༒Pɨυֆ༒Mᗩɳᗩƙɨʅ,UCsPQUCWRjWWGFvdyhjNLJ2A
551,Joshua Pavelko,UCEcc1trh3yFCcXHA1YQOezw


In [15]:
new_df.to_csv('youtube_channel_ids.csv')

In [47]:
new_df_1 = pd.read_csv('youtube_channel_ids.csv', index_col='Unnamed: 0')

In [48]:
new_df_1

,channel_title,channel_id
0,DSN- Data Scientists Network(Data Science Nige...,UCTi2xuVi9CLx9QryIJYLbaQ
1,freeCodeCamp.org,UC8butISFwT-Wl7EV0hUK0BQ
2,codebasics,UCh9nVJoWXmFb7sLApWGcLPQ
3,Steve Brunton,UCm5mt-A4w61lknZ9lCsZtBw
4,Data Professor,UCV8e2g4IWQqK71bbzGDEI4Q
...,...,...
548,Tony Games,UC9SDde41KHNC-VGw0qJ57mQ
549,Sheldon,UCtE0e8G-9zjwrQrug3Mra_A
550,DɾJØҽ༒Pɨυֆ༒Mᗩɳᗩƙɨʅ,UCsPQUCWRjWWGFvdyhjNLJ2A
551,Joshua Pavelko,UCEcc1trh3yFCcXHA1YQOezw


In [ ]:
yt_channe_ids_1 = df_2.channel_id.tolist()
print(yt_channe_ids_1)

In [27]:
yt_channe_ids_2 = df_1.channel_id.tolist()
print(len(yt_channe_ids_2))

488


In [ ]:
channel_ids = new_df.channel_id.tolist() # getting the ids of all the data science channels gathered
print(channel_ids)

In [7]:
def get_channel_stats(youtube, channel_ids):
       """ This function get the channels stats with the channel ids passed as an *args

       Args:
           youtube (Youtube Api): Your Youtube build Api V3
           channel_ids (str): list of ids or an id

       Returns:
           json data for all the ids or id: Output
       """       
     data = []
     for id in channel_ids: # iterating over the list of ids passed as the secon arguments
       try:
              request = youtube.channels().list(
                     part="snippet,contentDetails,statistics",
                     maxResults=50,
                     id = id )
              response = request.execute()
    
              for i in range(len(response['items'])):
                     data_dict = dict(channel_name = response['items'][i]['snippet']['title'],
                                      subscribers = response['items'][i]['statistics']['subscriberCount'],
                                      views = response['items'][i]['statistics']['viewCount'],
                                      videos = response['items'][i]['statistics']['videoCount'],
                                      published_date = response['items'][i]['snippet']['publishedAt'])
                     data.append(data_dict)
    
              next_page_token = response.get('nextPageToken')
              more_pages = True

              while more_pages:
                     if next_page_token is None:
                            more_pages = False
                     else:
                            request = youtube.channels().list(
                                   part="snippet,contentDetails,statistics",
                                   maxResults=50,
                                   id= id,
                                   pageToken = next_page_token)
                            response = request.execute()

                     for i in range(len(response['items'])):
                            data_dict = dict(channel_name = response['items'][i]['snippet']['title'],
                                             subscribers = response['items'][i]['statistics']['subscriberCount'],
                                             views = response['items'][i]['statistics']['viewCount'],
                                             videos = response['items'][i]['statistics']['videoCount'],
                                             published_date = response['items'][i]['snippet']['publishedAt'])
                            data.append(data_dict)
                     next_page_token = response.get('nextPageToken')
       except KeyError as ke: # this ignores keyerror because some channels information might unavailabe and to continue the programm
              print(ke)
              pass

     return data

In [ ]:
new_ids = [ids for ids in channel_ids]
new_ids

In [21]:
json_data = get_channel_stats(youtube_2, channel_ids)

'items'
'items'


In [24]:
print(len(json_data))

1850


In [23]:
file_name = 'full_yt_channels_stats.json'
with open(file_name, 'w') as file:
    json.dump(json_data, file)

In [21]:
file_name = 'first_yt_channels_stats.json'
with open(file_name, 'w') as file:
    json.dump(json_data, file)

In [32]:
file_name = 'second_yt_channels_stats.json'
with open(file_name, 'w') as file:
    json.dump(json_data, file)

In [33]:
with open('first_yt_channels_stats.json') as file:
    channel_stats_data_1 = json.load(file)

In [34]:
with open('second_yt_channels_stats.json') as file:
    channel_stats_data_2 = json.load(file)

In [25]:
with open('full_yt_channels_stats.json') as file:
    full_channel_stats_data = json.load(file)

In [26]:
full_channel_stats_df = pd.DataFrame(full_channel_stats_data)
full_channel_stats_df.head()

,channel_name,subscribers,views,videos,published_date
0,DSN- Data Scientists Network(Data Science Nige...,10000,707922,278,2017-09-13T16:36:47Z
1,DSN- Data Scientists Network(Data Science Nige...,10000,707922,278,2017-09-13T16:36:47Z
2,freeCodeCamp.org,6150000,412434264,1341,2014-12-16T21:18:48Z
3,freeCodeCamp.org,6150000,412434264,1341,2014-12-16T21:18:48Z
4,codebasics,645000,42421146,539,2015-11-07T17:29:46Z


In [35]:
channels_stat_df_1 = pd.DataFrame(channel_stats_data_1)
channels_stat_df_2 = pd.DataFrame(channel_stats_data_2)

In [28]:
# Merging the new gathered channel stats to their respective ids and title
merged_channel_stats_df = full_channel_stats_df.merge(new_df, left_on='channel_name', right_on='channel_title', how='left')

In [29]:
merged_channel_stats_df

,channel_name,subscribers,views,videos,published_date,channel_title,channel_id
0,DSN- Data Scientists Network(Data Science Nige...,10000,707922,278,2017-09-13T16:36:47Z,DSN- Data Scientists Network(Data Science Nige...,UCTi2xuVi9CLx9QryIJYLbaQ
1,DSN- Data Scientists Network(Data Science Nige...,10000,707922,278,2017-09-13T16:36:47Z,DSN- Data Scientists Network(Data Science Nige...,UCTi2xuVi9CLx9QryIJYLbaQ
2,freeCodeCamp.org,6150000,412434264,1341,2014-12-16T21:18:48Z,freeCodeCamp.org,UC8butISFwT-Wl7EV0hUK0BQ
3,freeCodeCamp.org,6150000,412434264,1341,2014-12-16T21:18:48Z,freeCodeCamp.org,UC8butISFwT-Wl7EV0hUK0BQ
4,codebasics,645000,42421146,539,2015-11-07T17:29:46Z,codebasics,UCh9nVJoWXmFb7sLApWGcLPQ
...,...,...,...,...,...,...,...
2213,DɾJØҽ༒Pɨυֆ༒Mᗩɳᗩƙɨʅ,3740,52669,7,2014-02-17T20:28:41Z,DɾJØҽ༒Pɨυֆ༒Mᗩɳᗩƙɨʅ,UCsPQUCWRjWWGFvdyhjNLJ2A
2214,Joshua Pavelko,0,0,0,2013-01-27T22:51:02Z,Joshua Pavelko,UCEcc1trh3yFCcXHA1YQOezw
2215,Joshua Pavelko,0,0,0,2013-01-27T22:51:02Z,Joshua Pavelko,UCEcc1trh3yFCcXHA1YQOezw
2216,Getloose360,15,0,0,2007-09-29T17:00:59Z,Getloose360,UCRT5dsm8Sbzlmd4S8ueyZqg


In [32]:
# Dropping the duplicates
merged_channel_stats_df.drop_duplicates(inplace=True)
merged_channel_stats_df

,channel_name,subscribers,views,videos,published_date,channel_title,channel_id
0,DSN- Data Scientists Network(Data Science Nige...,10000,707922,278,2017-09-13T16:36:47Z,DSN- Data Scientists Network(Data Science Nige...,UCTi2xuVi9CLx9QryIJYLbaQ
2,freeCodeCamp.org,6150000,412434264,1341,2014-12-16T21:18:48Z,freeCodeCamp.org,UC8butISFwT-Wl7EV0hUK0BQ
4,codebasics,645000,42421146,539,2015-11-07T17:29:46Z,codebasics,UCh9nVJoWXmFb7sLApWGcLPQ
6,Steve Brunton,215000,11902956,379,2011-10-07T15:57:41Z,Steve Brunton,UCm5mt-A4w61lknZ9lCsZtBw
8,Data Professor,132000,3388292,290,2019-08-17T15:59:56Z,Data Professor,UCV8e2g4IWQqK71bbzGDEI4Q
...,...,...,...,...,...,...,...
2208,Tony Games,0,0,0,2012-02-03T18:59:46Z,Tony Games,UC9SDde41KHNC-VGw0qJ57mQ
2210,Sheldon,0,0,0,2012-09-20T16:36:58Z,Sheldon,UCtE0e8G-9zjwrQrug3Mra_A
2212,DɾJØҽ༒Pɨυֆ༒Mᗩɳᗩƙɨʅ,3740,52669,7,2014-02-17T20:28:41Z,DɾJØҽ༒Pɨυֆ༒Mᗩɳᗩƙɨʅ,UCsPQUCWRjWWGFvdyhjNLJ2A
2214,Joshua Pavelko,0,0,0,2013-01-27T22:51:02Z,Joshua Pavelko,UCEcc1trh3yFCcXHA1YQOezw


In [33]:
merged_channel_stats_df.to_csv('complete-ds-yt-channel-data.csv')

## Cleaning

In [9]:
df = pd.read_csv('complete-ds-yt-channel-data.csv')

In [10]:
df.head()

,Unnamed: 0,channel_name,subscribers,views,videos,published_date,channel_title,channel_id
0,0,DSN- Data Scientists Network(Data Science Nige...,10000,707922,278,2017-09-13T16:36:47Z,DSN- Data Scientists Network(Data Science Nige...,UCTi2xuVi9CLx9QryIJYLbaQ
1,2,freeCodeCamp.org,6150000,412434264,1341,2014-12-16T21:18:48Z,freeCodeCamp.org,UC8butISFwT-Wl7EV0hUK0BQ
2,4,codebasics,645000,42421146,539,2015-11-07T17:29:46Z,codebasics,UCh9nVJoWXmFb7sLApWGcLPQ
3,6,Steve Brunton,215000,11902956,379,2011-10-07T15:57:41Z,Steve Brunton,UCm5mt-A4w61lknZ9lCsZtBw
4,8,Data Professor,132000,3388292,290,2019-08-17T15:59:56Z,Data Professor,UCV8e2g4IWQqK71bbzGDEI4Q


In [12]:
df = df.drop('Unnamed: 0', axis=1)

In [13]:
df.dtypes

channel_name      object
subscribers        int64
views              int64
videos             int64
published_date    object
channel_title     object
channel_id        object
dtype: object

In [14]:
df['subscribers'] = pd.to_numeric(df['subscribers'])
df['views'] = pd.to_numeric(df['views'])
df['videos'] = pd.to_numeric(df['videos'])
df['published_date'] = pd.to_datetime(df['published_date'])
df.dtypes

channel_name                   object
subscribers                     int64
views                           int64
videos                          int64
published_date    datetime64[ns, UTC]
channel_title                  object
channel_id                     object
dtype: object

In [15]:
df_copy = df.copy()

In [62]:
from datetime import date, datetime, timedelta
from dateutil import relativedelta
import numpy as np

In [17]:
now = datetime.now()

In [18]:
df_copy['current_date'] = now

In [19]:
df_copy.dtypes

channel_name                   object
subscribers                     int64
views                           int64
videos                          int64
published_date    datetime64[ns, UTC]
channel_title                  object
channel_id                     object
current_date           datetime64[ns]
dtype: object

In [20]:
df_copy.head()

,channel_name,subscribers,views,videos,published_date,channel_title,channel_id,current_date
0,DSN- Data Scientists Network(Data Science Nige...,10000,707922,278,2017-09-13 16:36:47+00:00,DSN- Data Scientists Network(Data Science Nige...,UCTi2xuVi9CLx9QryIJYLbaQ,2022-08-27 18:17:15.195041
1,freeCodeCamp.org,6150000,412434264,1341,2014-12-16 21:18:48+00:00,freeCodeCamp.org,UC8butISFwT-Wl7EV0hUK0BQ,2022-08-27 18:17:15.195041
2,codebasics,645000,42421146,539,2015-11-07 17:29:46+00:00,codebasics,UCh9nVJoWXmFb7sLApWGcLPQ,2022-08-27 18:17:15.195041
3,Steve Brunton,215000,11902956,379,2011-10-07 15:57:41+00:00,Steve Brunton,UCm5mt-A4w61lknZ9lCsZtBw,2022-08-27 18:17:15.195041
4,Data Professor,132000,3388292,290,2019-08-17 15:59:56+00:00,Data Professor,UCV8e2g4IWQqK71bbzGDEI4Q,2022-08-27 18:17:15.195041


In [28]:
# Stripping of the timezones from the dates
df_copy['published_date'] = df_copy['published_date'].dt.strftime('%Y-%m-%d')

In [30]:
df_copy['current_date'] = df_copy['current_date'].dt.strftime('%Y-%m-%d')

In [35]:
# Converting back to datetime
df_copy[['published_date', 'current_date']] = df_copy[['published_date', 'current_date']].astype('datetime64')

In [36]:
df_copy.dtypes

channel_name              object
subscribers                int64
views                      int64
videos                     int64
published_date    datetime64[ns]
channel_title             object
channel_id                object
current_date      datetime64[ns]
dtype: object

In [67]:
df_copy['channel_age_months'] =((df_copy['current_date'] - df_copy['published_date'])/np.timedelta64(1, 'M'))
df_copy['channel_age_months'] = df_copy['channel_age_months'].astype('int')

In [68]:
df_copy.dtypes

channel_name                  object
subscribers                    int64
views                          int64
videos                         int64
published_date        datetime64[ns]
channel_title                 object
channel_id                    object
current_date          datetime64[ns]
channel_age_months             int32
dtype: object

In [69]:
df_copy.head()

,channel_name,subscribers,views,videos,published_date,channel_title,channel_id,current_date,channel_age_months
0,DSN- Data Scientists Network(Data Science Nige...,10000,707922,278,2017-09-13,DSN- Data Scientists Network(Data Science Nige...,UCTi2xuVi9CLx9QryIJYLbaQ,2022-08-27,59
1,freeCodeCamp.org,6150000,412434264,1341,2014-12-16,freeCodeCamp.org,UC8butISFwT-Wl7EV0hUK0BQ,2022-08-27,92
2,codebasics,645000,42421146,539,2015-11-07,codebasics,UCh9nVJoWXmFb7sLApWGcLPQ,2022-08-27,81
3,Steve Brunton,215000,11902956,379,2011-10-07,Steve Brunton,UCm5mt-A4w61lknZ9lCsZtBw,2022-08-27,130
4,Data Professor,132000,3388292,290,2019-08-17,Data Professor,UCV8e2g4IWQqK71bbzGDEI4Q,2022-08-27,36


In [52]:
overall_merged.to_csv('ds_youtube_channel_data.csv')

Further Analysis on Tableau 
[Click here for the Analysis](https://public.tableau.com/views/Youtube-Analysis/Dashboard1?:language=en-US&:display_count=n&:origin=viz_share_link)